<a href="https://colab.research.google.com/github/Tj-dev-py/Speech-Commands-Classification/blob/main/Speech_Commands_(Model_Training_and_Testing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np

extracted_features_df = pd.read_pickle('/content/drive/MyDrive/Speech Commands Audio Classification /Extracted Features/extracted_features.pkl')
display(extracted_features_df.head(), extracted_features_df.shape)

,feature,class
0,"[-588.5346, 165.72563, -19.97634, 25.505146, -...",down
1,"[-536.72925, 196.67656, -42.150036, 28.270063,...",down
2,"[-535.14197, 170.9807, -19.985088, 34.06252, 1...",down
3,"[-488.70197, 229.5985, -70.07708, 33.4808, 10....",down
4,"[-546.6566, 166.20728, -41.497158, 28.023365, ...",down


(27571, 2)

In [2]:
# Splitting the dataset into independent and dependent variables
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [3]:
display(X[:2], X.shape, y[:2], y.shape)

array([[-5.88534607e+02,  1.65725632e+02, -1.99763393e+01,
         2.55051460e+01, -8.37503338e+00, -1.41040030e+01,
         3.90248394e+00,  3.24194551e+00, -2.03755550e+01,
         2.42028618e+00, -5.17369032e+00, -9.40413761e+00,
         3.13723063e+00, -1.41327658e+01,  3.55066752e+00,
         4.87875462e+00, -5.70190096e+00, -6.24347830e+00,
         7.68477774e+00, -4.26375151e+00, -8.27132893e+00,
         3.07360935e+00, -4.54137945e+00,  1.39910445e-01,
        -5.86316681e+00,  3.76577830e+00,  1.65205812e+00,
        -3.76900935e+00,  3.39766324e-01, -8.07228684e-01,
        -7.10546672e-02, -5.20493746e+00,  2.46700001e+00,
        -3.09450507e+00, -1.14392805e+00,  6.96672082e-01,
        -3.16858196e+00, -2.96967272e-02, -1.43387508e+00,
        -3.32503676e-01],
       [-5.36729248e+02,  1.96676559e+02, -4.21500359e+01,
         2.82700634e+01, -3.38422942e+00, -2.20895042e+01,
         4.80113268e+00,  4.84922838e+00, -2.21200886e+01,
         5.95539284e+00, -1.55

(27571, 40)

array(['down', 'down'], dtype='<U5')

(27571,)

In [4]:
# Label encoding
y = np.array(pd.get_dummies(y))

In [5]:
X.shape, y.shape

((27571, 40), (27571, 20))

In [6]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [7]:
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(22056, 40)

(5515, 40)

(22056, 20)

(5515, 20)

In [8]:
y[1]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)

In [9]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [10]:
# import libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras import regularizers
from sklearn import metrics

In [11]:
# Number of classes
num_labels = y.shape[1]

In [12]:
num_labels

20

In [13]:
# setting a callback
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,

)

In [14]:
# Creating the neural network
model = Sequential([
                    # First layer
                    Dense(700, input_shape=(40, ), activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
                    Dropout(.5),

                    # Final Layer
                    Dense(num_labels, activation='softmax')
])

In [15]:
# Model's summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 700)               28700     
                                                                 
 dropout (Dropout)           (None, 700)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                14020     
                                                                 
Total params: 42,720
Trainable params: 42,720
Non-trainable params: 0
_________________________________________________________________


In [16]:
# compiling the model

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [17]:
num_epochs = 100
num_batch_size = 32

history = model.fit(X_train, y_train,
          batch_size=num_batch_size,
          epochs=num_epochs,
          validation_data=(X_test, y_test),
          callbacks=[early_stopping]
          )

Epoch 1/100
690/690 [==============================] - 3s 4ms/step - loss: 7.4425 - accuracy: 0.3129 - val_loss: 1.4057 - val_accuracy: 0.5422
Epoch 2/100
690/690 [==============================] - 2s 3ms/step - loss: 1.4766 - accuracy: 0.5098 - val_loss: 1.0910 - val_accuracy: 0.6470
Epoch 3/100
690/690 [==============================] - 2s 3ms/step - loss: 1.2453 - accuracy: 0.5747 - val_loss: 0.9241 - val_accuracy: 0.6972
Epoch 4/100
690/690 [==============================] - 2s 3ms/step - loss: 1.1239 - accuracy: 0.6139 - val_loss: 0.8365 - val_accuracy: 0.7179
Epoch 5/100
690/690 [==============================] - 2s 3ms/step - loss: 1.0285 - accuracy: 0.6456 - val_loss: 0.7558 - val_accuracy: 0.7626
Epoch 6/100
690/690 [==============================] - 2s 3ms/step - loss: 0.9639 - accuracy: 0.6646 - val_loss: 0.6841 - val_accuracy: 0.7753
Epoch 7/100
690/690 [==============================] - 2s 3ms/step - loss: 0.9237 - accuracy: 0.6827 - val_loss: 0.6577 - val_accuracy: 0.7880

In [18]:
tf.keras.models.save_model(model,'speech_commands_model.hdf5')

In [35]:
model = tf.keras.models.load_model('/content/speech_commands_model.hdf5')

In [20]:
test_accuracy = model.evaluate(X_test, y_test)
print(test_accuracy)

173/173 [==============================] - 0s 1ms/step - loss: 0.4887 - accuracy: 0.8464
[0.48865771293640137, 0.8464188575744629]


In [29]:
filename = '/content/drive/MyDrive/Speech Commands Audio Classification /augmented_dataset/zero/zero_102.wav'

import librosa

def feature_extractor(file):
  audio, sample_rate = librosa.load(file, res_type='kaiser_fast')
  mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

  return mfccs_scaled_features

In [30]:
prediction_feature = feature_extractor(filename)

In [31]:
# before reshaping
prediction_feature.shape

(40,)

In [32]:
# after reshaping
prediction_feature.reshape(1, -1).shape

(1, 40)

In [33]:
prediction_feature = prediction_feature.reshape(1, -1)

In [34]:
np.argmax(model.predict(prediction_feature), axis=-1)

array([19])

Our model was correct in its prediction

In [38]:
import IPython.display as ipd
ipd.Audio('/content/drive/MyDrive/Speech Commands Audio Classification /augmented_dataset/seven/seven_1084.wav')